<a href="https://colab.research.google.com/github/AKShaffer/Group-2---Generative-AI-Tools-for-Drug-Repurposing-for-PD/blob/main/QSAR_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip -q install rdkit
!pip -q install plotly
import plotly.io as pio
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski
import pandas as pd


import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from IPython.display import HTML

import os
import io
#import sys
from google.colab import drive
drive.mount('/content/drive')
#import cv2
from io import BytesIO
import csv

# Change this to your file path in your Google Drive
csv_path = '/content/drive/My Drive/VAE_GAN_Colab_Notebooks/Data/consolidated_dockscores_with descriptors.csv'
df = pd.read_csv(csv_path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,smiles,target,docking_score,method,alphabet,atom_count,has_ring,method_target,qed,sas,logP,molWt
0,CCCC(C)C(CC)C(C)C,DRD2,-4.550000e+00,GAN,CCCCCCCCCCC,11,False,GAN_DRD2,0.561797,3.157874,4.1048,156.313
1,CCCC(C)C1C(O)C[SH]1Cl,DRD2,9.753651e+07,GAN,CCCCCCCOCSHCl,13,True,GAN_DRD2,0.664419,5.471001,2.3207,196.743
2,CCCC(C)C(SC)C(C)O,DRD2,-4.606000e+00,GAN,CCCCCCSCCCO,11,False,GAN_DRD2,0.694405,4.021779,2.5350,176.325
3,CC=CCCO.CCC,DRD2,-4.276000e+00,GAN,CCCCCOCCC,9,False,GAN_DRD2,0.568822,2.623218,2.3612,130.231
4,CCCCC(C)CCC,DRD2,-4.284000e+00,GAN,CCCCCCCCC,9,False,GAN_DRD2,0.530272,2.267231,3.6128,128.259


In [13]:
descriptors_list = []
for smiles in df['smiles'].to_list():
    mol = Chem.MolFromSmiles(smiles)
    if mol:  # Check if molecule object is successfully created
        descriptors = {
            'TPSA': Descriptors.TPSA(mol),  # Topological Polar Surface Area (TPSA)
            'Flexibility': Lipinski.NumRotatableBonds(mol),  # Flexibility (Rotatable bonds)
            '#RB': Lipinski.NumRotatableBonds(mol),  # Number of Rotatable Bonds (#RB)
            'TotalFormalCharge': Chem.rdmolops.GetFormalCharge(mol),  # Total Formal Charge
            'NumHAcceptors': Lipinski.NumHAcceptors(mol),  # Number of H-Acceptors
            'NumHDonors': Lipinski.NumHDonors(mol),  # Number of H-Donors
            'RingCount': Lipinski.RingCount(mol)  # Ring Count
        }
        descriptors_list.append(descriptors)

# Create the descriptors DataFrame from the list of descriptors.
descriptors_df = pd.DataFrame(descriptors_list)

# Filter out rows with blank 'qed' values from the original DataFrame.
df_filtered = df[df['qed'].notna()]

# Combine the filtered DataFrame and the descriptors DataFrame.
combined_df = pd.concat([df_filtered, descriptors_df], axis=1)

# Remove rows with NaN values.
combined_df.dropna(inplace=True)

# Save the combined DataFrame to a CSV file.
combined_df.to_csv('smiles_with_descriptors.csv', index=False)

combined_df.head()

[23:40:32] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11 12 13 14 15 16 22
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] Explicit valence for atom # 1 O, 3, is greater than permitted
[23:40:32] Explicit valence for atom # 1 O, 3, is greater than permitted
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 1 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 1 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] Can't kekulize mol.  Unkekulized atoms: 8 9 11 12 15
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring atom 0 marked aromatic
[23:40:32] non-ring a

,smiles,target,docking_score,method,alphabet,atom_count,has_ring,method_target,qed,sas,logP,molWt,TPSA,Flexibility,#RB,TotalFormalCharge,NumHAcceptors,NumHDonors,RingCount
0,CCCC(C)C(CC)C(C)C,DRD2,-4.550000e+00,GAN,CCCCCCCCCCC,11.0,False,GAN_DRD2,0.561797,3.157874,4.1048,156.313,0.00,5.0,5.0,0.0,0.0,0.0,0.0
1,CCCC(C)C1C(O)C[SH]1Cl,DRD2,9.753651e+07,GAN,CCCCCCCOCSHCl,13.0,True,GAN_DRD2,0.664419,5.471001,2.3207,196.743,20.23,3.0,3.0,0.0,1.0,2.0,1.0
2,CCCC(C)C(SC)C(C)O,DRD2,-4.606000e+00,GAN,CCCCCCSCCCO,11.0,False,GAN_DRD2,0.694405,4.021779,2.5350,176.325,20.23,5.0,5.0,0.0,2.0,1.0,0.0
3,CC=CCCO.CCC,DRD2,-4.276000e+00,GAN,CCCCCOCCC,9.0,False,GAN_DRD2,0.568822,2.623218,2.3612,130.231,20.23,2.0,2.0,0.0,1.0,1.0,0.0
4,CCCCC(C)CCC,DRD2,-4.284000e+00,GAN,CCCCCCCCC,9.0,False,GAN_DRD2,0.530272,2.267231,3.6128,128.259,0.00,5.0,5.0,0.0,0.0,0.0,0.0


In [14]:
train_df_DRD2 = combined_df[combined_df['method_target'] == 'APPROVED_DRD2']
test_df_GAN_DRD2 = combined_df[combined_df['method_target'] == 'GAN_DRD2']
test_df_sGPT_DRD2 = combined_df[combined_df['method_target'] == 'sGPT_DRD2']

train_df_SNCA = combined_df[combined_df['method_target'] == 'APPROVED_SNCA']
test_df_GAN_SNCA = combined_df[combined_df['method_target'] == 'GAN_SNCA']
test_df_sGPT_SNCA = combined_df[combined_df['method_target'] == 'sGPT_SNCA']
train_df_DRD2.head()

,smiles,target,docking_score,method,alphabet,atom_count,has_ring,method_target,qed,sas,logP,molWt,TPSA,Flexibility,#RB,TotalFormalCharge,NumHAcceptors,NumHDonors,RingCount
376,CC(CC1=CC=CC=C1)N,DRD2,-5.113,APPROVED,CCCCCCCCCN,10.0,True,APPROVED_DRD2,0.654216,1.838588,1.57630,135.210,6.48,4.0,4.0,0.0,4.0,0.0,4.0
377,C1CN(CCN1CCC2=C(C=C3C(=C2)CC(=O)N3)Cl)C4=NSC5=...,DRD2,-8.011,APPROVED,CCNCCNCCCCCCCCCCONClCNSCCCCCC,29.0,True,APPROVED_DRD2,0.707487,2.611506,3.80900,412.946,47.86,4.0,4.0,0.0,4.0,2.0,3.0
378,CCNC(=O)N(CCCN(C)C)C(=O)[C@@H]1C[C@H]2[C@@H](C...,DRD2,-7.202,APPROVED,CCNCONCCCNCCCOCHCCHCHCCCNCCCCCCNCCCC,36.0,True,APPROVED_DRD2,0.604938,4.016443,3.19390,451.615,118.21,4.0,4.0,0.0,6.0,3.0,8.0
379,CCCN(CCC)CCC1=C2CC(=O)NC2=CC=C1,DRD2,-6.560,APPROVED,CCCNCCCCCCCCCONCCCC,19.0,True,APPROVED_DRD2,0.817332,2.424310,2.84570,260.381,43.70,0.0,0.0,0.0,3.0,2.0,4.0
380,CC1=CC=CC=C1N2CCN(CC2)CCC3=NN=C4N3CCCC4,DRD2,-9.132,APPROVED,CCCCCCCNCCNCCCCCNNCNCCCC,24.0,True,APPROVED_DRD2,0.864228,2.218535,2.28752,325.460,39.72,4.0,4.0,0.0,4.0,1.0,4.0


In [15]:
# Create a dropdown widget for column selection
drop_columns=['smiles', 'target', 'method', 'alphabet', 'has_ring', 'method_target']
column_options = train_df_DRD2.drop(drop_columns, axis=1).columns[0]
column_options = train_df_DRD2.drop(drop_columns, axis=1).columns[0]

column_dropdown = widgets.Dropdown(
    options=train_df_DRD2.drop(drop_columns, axis=1),  # Use available columns as options
    value=column_options,  # Set default value to to first column
    description='Target Column:',
    layout=widgets.Layout(width='auto')  # Allow dropdown to adjust width
)

# Apply CSS styling to the description label
style = """
<style>
.widget-label {
    min-width: 150px; /* Adjust the value as needed */
}
</style>
"""
display(HTML(style))

# Create a button to trigger model fitting
fit_button = widgets.Button(description="Fit Models")

# Create an HBox to arrange the dropdown and button horizontally
controls = widgets.HBox([column_dropdown, fit_button])
display(controls)

# Function to update target variables based on dropdown selection
def update_targets(change=None):
    global DRD2_y, SNCA_y, DRD2_x, SNCA_x, selected_column  # Access global variables
    selected_column = change['new'] if change else column_dropdown.value # Get selected column from dropdown

    # Get the index of the selected column
    index = train_df_DRD2.columns.get_loc(selected_column)

    # Select all columns from the selected column to the end
    DRD2_x = train_df_DRD2.drop(drop_columns, axis=1)  # Use available columns as options
    SNCA_x = train_df_SNCA.drop(drop_columns, axis=1) # Use available columns as options
    # DRD2_x = train_df_DRD2.iloc[:, index:]
    # SNCA_x = train_df_SNCA.iloc[:, index:]

    # Select the target column for DRD2 and SNCA
    DRD2_y = train_df_DRD2[selected_column]
    SNCA_y = train_df_SNCA[selected_column]

# Function to fit the models when the button is clicked
def fit_models(b):
    DRD2_model.fit(DRD2_x, DRD2_y)
    SNCA_model.fit(SNCA_x, SNCA_y)
    print(f'Models fitted for {selected_column}.')

# Observe dropdown changes and call update_targets function
column_dropdown.observe(update_targets, names='value')

# Attach fit_models function to button click event
fit_button.on_click(fit_models)

# Create and fit separate models for DRD2 and SNCA
#DRD2_model=rf()
#SNCA_model=rf()

# Initial model fitting using default 'qed' column
update_targets()  # Initialize target variables based on initial dropdown selection

# Create and fit separate models for DRD2 and SNCA
DRD2_model=rf()
SNCA_model=rf()

Models fitted for qed.


In [17]:
GAN_DRD2_x_test=test_df_GAN_DRD2.drop(drop_columns,axis=1)
GAN_DRD2_y_pred=DRD2_model.predict(GAN_DRD2_x_test)

GAN_DRD2_y_test=test_df_GAN_DRD2[selected_column]

print(f'GAN DRD2 Test set R2: {r2_score(GAN_DRD2_y_test,GAN_DRD2_y_pred):.2f}')

GAN_SNCA_x_test=test_df_GAN_SNCA.drop(drop_columns,axis=1)
GAN_SNCA_y_pred=SNCA_model.predict(GAN_SNCA_x_test)

GAN_SNCA_y_test=test_df_GAN_SNCA[selected_column]

print(f'GAN SNCA Test set R2: {r2_score(GAN_SNCA_y_test,GAN_SNCA_y_pred):.2f}')

sGPT_DRD2_x_test=test_df_sGPT_DRD2.drop(drop_columns,axis=1)
sGPT_DRD2_y_pred=DRD2_model.predict(sGPT_DRD2_x_test)

sGPT_DRD2_y_test=test_df_sGPT_DRD2[selected_column]

print(f'sGPT DRD2 Test set R2: {r2_score(sGPT_DRD2_y_test,sGPT_DRD2_y_pred):.2f}')

sGPT_SNCA_x_test=test_df_sGPT_SNCA.drop(drop_columns,axis=1)
sGPT_SNCA_y_pred=SNCA_model.predict(sGPT_SNCA_x_test)

sGPT_SNCA_y_test=test_df_sGPT_SNCA[selected_column]

print(f'sGPT SNCA Test set R2: {r2_score(sGPT_SNCA_y_test,sGPT_SNCA_y_pred):.2f}')


GAN DRD2 Test set R2: 0.99
GAN SNCA Test set R2: -0.30
sGPT DRD2 Test set R2: 0.78
sGPT SNCA Test set R2: -9.72


In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=GAN_DRD2_y_test, y=GAN_DRD2_y_pred, mode='markers', name='GAN_DRD2'))
fig.add_trace(go.Scatter(x=sGPT_DRD2_y_test, y=sGPT_DRD2_y_pred, mode='markers', name='sGPT_DRD2'))
fig.add_trace(go.Scatter(x=GAN_SNCA_y_test, y=GAN_SNCA_y_pred, mode='markers', name='GAN_SNCA'))
fig.add_trace(go.Scatter(x=sGPT_SNCA_y_test, y=sGPT_SNCA_y_pred, mode='markers', name='sGPT_SNCA'))

# Calculate min and max values for x and y
x_min = round(min(min(GAN_DRD2_y_test), min(sGPT_DRD2_y_test), min(GAN_SNCA_y_test), min(sGPT_SNCA_y_test)))
x_max = round(max(max(GAN_DRD2_y_test), max(sGPT_DRD2_y_test), max(GAN_SNCA_y_test), max(sGPT_SNCA_y_test)))
y_min = round(min(min(GAN_DRD2_y_pred), min(sGPT_DRD2_y_pred), min(GAN_SNCA_y_pred), min(sGPT_SNCA_y_pred)))
y_max = round(max(max(GAN_DRD2_y_pred), max(sGPT_DRD2_y_pred), max(GAN_SNCA_y_pred), max(sGPT_SNCA_y_pred)))

fig.update_layout(
    title= f'{selected_column} Prediction',
    xaxis_title=f'{selected_column} True',
    yaxis_title=f'{selected_column} Pred',
    shapes=[dict(type='line', x0=x_min, y0=y_min, x1=x_max, y1=y_max, line=dict(color='red', width=2))],
    title_x=0.5  # Center the title
)

# Create widgets for file names
csv_file_name = widgets.Text(value=f'{selected_column}_graph_data.csv', description='CSV File Name:')
plot_file_name = widgets.Text(value=f'{selected_column}_prediction_plot.png', description='Plot File Name:')

# Create buttons for saving
save_csv_button = widgets.Button(description="Save CSV")
save_plot_button = widgets.Button(description="Save Plot")

fig.show()

# Display widgets
display(csv_file_name, plot_file_name, save_csv_button, save_plot_button)

# Define save functions
def save_csv(b):
    all_data = []  # Initialize all_data here
    for trace in fig.data:
        xaxis_title = fig.layout.xaxis.title.text
        yaxis_title = fig.layout.yaxis.title.text
        trace_data = pd.DataFrame({xaxis_title: trace.x, yaxis_title: trace.y, 'method': [trace.name] * len(trace.x)})
        all_data.append(trace_data)
    all_data = pd.concat(all_data)
    all_data.to_csv(csv_file_name.value, index=False)
    print(f"CSV saved as {csv_file_name.value}")

def save_plot(b):
    #pio.kaleido.scope.default_format = "png"  # Explicitly set the format
    fig.write_image(plot_file_name.value, engine="kaleido")
    print(f"Plot saved as {plot_file_name.value}")

# Attach save functions to buttons
save_csv_button.on_click(save_csv)
save_plot_button.on_click(save_plot)

Text(value='qed_graph_data.csv', description='CSV File Name:')

Text(value='qed_prediction_plot.png', description='Plot File Name:')

Button(description='Save CSV', style=ButtonStyle())

Button(description='Save Plot', style=ButtonStyle())

Plot saved as qed_prediction_plot.png
